
# Diferenciação Automática com Grafos Computacionais


### Imports

In [ ]:
from typing import Optional, Union, Any
from collections.abc import Iterable
from abc import ABC, abstractmethod
import numbers

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

### Classe NameManager

A classe NameManager provê uma forma conveniente de dar nomes intuitivos para tensores que resultam de operações. A idéia é tornar mais fácil para o usuário das demais classes qual operação gerou qual tensor. Ela provê os seguintes métodos públicos:

- reset(): reinicia o sistema de gestão de nomes.
- new(<basename>: str): retorna um nome único a partir do nome de base passado como argumento.
  
Como indicado no exemplo abaixo da classe, a idéia geral é que uma sequência de operações é feita, os nomes dos tensores sejam os nomes das operações seguidos de um número. Se forem feitas 3 operações de soma e uma de multiplicação, seus tensores de saída terão os nomes "add:0", "add:1", "add:2" e "prod:0".

In [ ]:

class NameManager:
    _counts = {}

    @staticmethod
    def reset():
        NameManager._counts = {}

    @staticmethod
    def _count(name):
        if name not in NameManager._counts:
            NameManager._counts[name] = 0
        count = NameManager._counts[name]
        return count

    @staticmethod
    def _inc_count(name):
        assert name in NameManager._counts, f'Name {name} is not registered.'
        NameManager._counts[name] += 1

    @staticmethod
    def new(name: str):
        count = NameManager._count(name)
        tensor_name = f"{name}:{count}"
        NameManager._inc_count(name)
        return tensor_name

# exemplo de uso
print(NameManager.new('add'))
print(NameManager.new('in'))
print(NameManager.new('add'))
print(NameManager.new('add'))
print(NameManager.new('in'))
print(NameManager.new('prod'))

NameManager.reset()

add:0
in:0
add:1
add:2
in:1
prod:0


### Classe Tensor

Classe `Tensor` representando um array multidimensional.

In [ ]:
def garante_numpy_array(arr):
  if not isinstance(arr,np.ndarray):
    return np.array(arr,dtype=float)
  else:
    return arr.astype(float) # astype() só cria uma copia se o tipo for diferente

class Tensor:
    """Classe representando um array multidimensional.

    Atributos:

    - _arr  (privado): dados internos do tensor como
        um array do numpy com 2 dimensões (ver Regras)

    - _parents (privado): lista de tensores que foram
        usados como argumento para a operação que gerou o
        tensor. Será vazia se o tensor foi inicializado com
        valores diretamente. Por exemplo, se o tensor foi
        resultado da operação a + b entre os tensores a e b,
        _parents = [a, b].

    - requires_grad (público): indica se devem ser
        calculados gradientes para o tensor ou não.

    - grad (público): Tensor representando o gradiente.

    """

    def __init__(self,
                 # Dados do tensor. Além dos tipos listados,
                 # arr também pode ser do tipo Tensor.
                 arr: Union[np.ndarray, list, numbers.Number, Any],
                 # Entradas da operacao que gerou o tensor.
                 # Deve ser uma lista de itens do tipo Tensor.
                 parents: list[Any] = [],
                 # se o tensor requer o calculo de gradientes ou nao
                 requires_grad: bool = True,
                 # nome do tensor
                 name: str = '',
                 # referência para um objeto do tipo Operation (ou
                 # subclasse) indicando qual operação gerou este
                 # tensor. Este objeto também possui um método
                 # para calcular a derivada da operação.
                 operation=None):

        self._arr = None
        if isinstance(arr, Tensor):
          self._arr = arr.numpy().copy()
        else:
          arr = garante_numpy_array(arr)

          if arr.ndim == 0:
            self._arr = arr.reshape(1,1)
          elif arr.ndim == 1:
            self._arr = arr.reshape(-1,1)
          elif arr.ndim == 2:
            self._arr = arr
          else:
            raise ValueError(f"Entrada com {arr.ndim} dimensões não é suportada para o Tensor.")

        self._parents = parents

        self.requires_grad = requires_grad

        self.operation = operation
        self.grad = None

        if name:
          self._name = name
        else:
          if not parents and operation is None:
            self._name = NameManager.new('in')


    def zero_grad(self):
        """Reinicia o gradiente com zero"""
        self.grad = Tensor(np.zeros_like(self._arr))

    def numpy(self):
        """Retorna o array interno"""
        return self._arr

    def __repr__(self):
        """Permite visualizar os dados do tensor como string"""
        return f"Tensor({self._arr}, name={self._name}, shape={self._arr.shape})"

    def backward(self, my_grad=None):
        """Método usado tanto iniciar o processo de
        diferenciação automática, quanto por um filho
        para enviar o gradiente do pai. No primeiro
        caso, o argumento my_grad não será passado.
        """

        # Caso seja o primeiro gradiente analisado
        if my_grad == None:
          self.grad = Tensor(np.ones_like(self._arr), requires_grad = False)
        else:
          if self.grad == None:
            # Caso seja um tensor inicial
            if not self._parents and self.operation is None:
              self.grad = Tensor(np.zeros_like(self._arr), name = 'in_grad')
            else:
              self.grad = Tensor(np.zeros_like(self._arr), name = my_grad._name)

          assert self.grad.numpy().shape == my_grad.numpy().shape
          self.grad._arr += my_grad.numpy()

        if self.operation == None:
          return

        parents_grads = self.operation.grad(self.grad,*self._parents)


        for i, parent in enumerate(self._parents):
          if parent.requires_grad:
            #  print(parents_grads[i])
            #  print('----------------------------------------------------------------\n')
             parent.backward(my_grad = parents_grads[i])


### Interface de  Operações

A classe abaixo define a interface que as operações devem implementar. Ela não precisa ser modificada, mas pode, caso queira.

In [ ]:

class Op(ABC):
    @abstractmethod
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando as entradas e
            retorna o tensor resultado."""

    @abstractmethod
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna os gradientes dos pais em como tensores.

        Arguments:

        - back_grad: Derivada parcial em relação à saída
            da operação backpropagada pelo filho.

        - args: variaveis de entrada da operacao (pais)
            como tensores.

        """


    # GARANTE QUE ARGS SAO TENSORES
    def _ts(self,args):
      tensor_elements = []
      for elemento in args:
        if not isinstance(elemento,Tensor):
          elemento_tensor = Tensor(arr=elemento,requires_grad=False)
          tensor_elements.append(elemento_tensor)
        else:
          tensor_elements.append(elemento)
      return tuple(tensor_elements)


### Implementação das Operações

Operações devem herdar de `Op` e implementar os métodos `__call__` e `grad`.

Pelo menos as seguintes operações devem ser implementadas:



In [ ]:

class Add(Op):
    """Add(a, b): a + b"""
    def __call__(self, *args, **kwargs) -> Tensor:
         """Realiza a operação usando os argumentos dados em args"""
        args = self._ts(args)
        assert args[0].numpy().shape == args[1].numpy().shape or \
           args[0].numpy().size == 1 or args[1].numpy().size == 1, \
           f"Shapes para Add devem ser iguais ou um deve ser escalar: {args[0].numpy().shape}, {args[1].numpy().shape}"

        result = args[0].numpy() + args[1].numpy()
        return Tensor(result,parents=args,name = NameManager.new('add'),operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
        return [Tensor(back_grad, name=NameManager.new('add_grad')),Tensor(back_grad, name=NameManager.new('add_grad'))] # No caso é backgrad * 1 pois eh uma soma. backgrad = grad do filho



# Instancia a classe. O objeto passa a poder ser usado como uma funcao
add = Add()

In [ ]:

class Sub(Op):
    """Sub(a, b): a - b"""
    def __call__(self, *args, **kwargs) -> Tensor:

        args = self._ts(args)
        assert args[0].numpy().shape == args[1].numpy().shape or \
           args[0].numpy().size == 1 or args[1].numpy().size == 1, \
           f"Shapes para Add devem ser iguais ou um deve ser escalar: {args[0].numpy().shape}, {args[1].numpy().shape}"

        result = args[0].numpy() - args[1].numpy()
        return Tensor(result,parents=args,name = NameManager.new('sub'),operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:

        return [Tensor(back_grad, name=NameManager.new('sub_grad')),Tensor(back_grad._arr * (-1), name=NameManager.new('sub_grad'))]


sub = Sub()

In [ ]:

class Prod(Op):
    """Prod(a, b): produto ponto a ponto de a e b ou produto escalar-tensor"""
    def __call__(self, *args, **kwargs) -> Tensor:
        args = self._ts(args)

        assert args[0].numpy().shape == args[1].numpy().shape or \
           args[0].numpy().size == 1 or args[1].numpy().size == 1, \
           f"Shapes para Add devem ser iguais ou um deve ser escalar: {args[0].numpy().shape}, {args[1].numpy().shape}"

        result = args[0].numpy() * args[1].numpy()
        return Tensor(result,parents=args,name = NameManager.new('prod'),operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:

        return [Tensor(self(back_grad,args[1]), name=NameManager.new('prod_grad')),Tensor(self(back_grad,args[0]), name=NameManager.new('prod_grad'))]


prod = Prod()

In [ ]:

class Sin(Op):
    """seno element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:

        args = self._ts(args)
        result = np.sin(args[0].numpy())
        return Tensor(result,parents=args,name = NameManager.new('sin'),operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
        result_sin_grad = np.cos(args[0]._arr) * back_grad._arr
        return [Tensor(result_sin_grad, name=NameManager.new('sin_grad'))]


sin = Sin()

In [ ]:

class Cos(Op):
    """cosseno element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:

        args = self._ts(args)
        result = np.cos(args[0].numpy())
        return Tensor(result,parents=args,name = NameManager.new('cos'),operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:

        result_cos_grad = np.sin(args[0]._arr) * (-1) * back_grad._arr
        return [Tensor(result_cos_grad, name=NameManager.new('cos_grad'))]



cos = Cos()

In [ ]:

class Sum(Op):
    """Retorna a soma dos elementos do tensor"""
    def __call__(self, *args, **kwargs) -> Tensor:

        args = self._ts(args)
        result = np.sum(args[0].numpy())
        return Tensor(result,parents=args,name = NameManager.new('my_sum'),operation=self)


    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        ones = Tensor(np.ones_like(args[0].numpy()))
        return [Tensor(prod(ones,back_grad), name=NameManager.new('my_sum_grad'))]


my_sum = Sum()

In [ ]:

class Mean(Op):
    """Retorna a média dos elementos do tensor"""
    def __call__(self, *args, **kwargs) -> Tensor:
        args = self._ts(args)
        result = np.mean(args[0].numpy())
        return Tensor(result,parents=args,name = NameManager.new('mean'),operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        n_elementos = args[0].numpy().size
        ones = Tensor(np.ones_like(args[0].numpy()))

        result_mean_grad = back_grad._arr / n_elementos

        return [Tensor(prod(ones,result_mean_grad), name=NameManager.new('mean_grad'))]


mean = Mean()

In [ ]:

class Square(Op):
    """Eleva cada elemento ao quadrado"""
    def __call__(self, *args, **kwargs) -> Tensor:
        args = self._ts(args)
        result = np.power(args[0].numpy(),2)
        return Tensor(result,parents=args,name = NameManager.new('square'),operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        result_square_grad = 2 * args[0].numpy() * back_grad._arr
        return [Tensor(result_square_grad,name = NameManager.new('square_grad'))]


square = Square()

In [ ]:

class MatMul(Op):
    """MatMul(A, B): multiplicação de matrizes

    C = A @ B
    de/dA = de/dc @ B^T
    de/dB = A^T @ de/dc

    """

    def __call__(self, *args, **kwargs) -> Tensor:
        args = self._ts(args)
        assert args[0].numpy().shape[1] == args[1].numpy().shape[0], \
            f"Shape incompatível para MatMul: {args[0].numpy().shape} @ {args[1].numpy().shape}"

        result = args[0].numpy() @ args[1].numpy()
        return Tensor(result, parents=args, name=NameManager.new('matmul'), operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        A = args[0]
        B = args[1]

        grad_A = self(back_grad._arr,B._arr.T)
        grad_B = self(A._arr.T,back_grad)

        return [Tensor(grad_A,name= NameManager.new('matmul_grad')), Tensor(grad_B,name= NameManager.new('matmul_grad'))]


matmul = MatMul()

In [ ]:

class Exp(Op):
    """Exponenciação element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:
        args = self._ts(args)
        result = np.exp(args[0]._arr)
        return Tensor(result, parents=args, name=NameManager.new('exp'), operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:

        result_exp_grad = np.exp(args[0].numpy()) * back_grad.numpy()

        return [Tensor(result_exp_grad, name = NameManager.new('exp_grad'))]

exp = Exp()

In [ ]:

class ReLU(Op):
    """ReLU element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:

        args = self._ts(args)
        result = np.maximum(0, args[0].numpy())
        return Tensor(result, parents=args, name=NameManager.new('relu'), operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:

        mask = args[0].numpy() > 0
        mask_float = mask.astype(float)

        result_relu_grad = back_grad.numpy() * mask_float

        return [Tensor(result_relu_grad,name = NameManager.new('relu_grad'))]

relu = ReLU()

In [ ]:

class Sigmoid(Op):
    """Sigmoid element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:

        args = self._ts(args)
        result = 1 / (1+ np.exp(-args[0].numpy()))
        return Tensor(result, parents=args, name=NameManager.new('sigmoid'), operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:

        y = self(args[0].numpy())
        result_sigmoid_grad = y.numpy() * (1 - y.numpy()) # y * (1 - y)
        result_sigmoid_grad = result_sigmoid_grad * back_grad.numpy()


        return [Tensor(result_sigmoid_grad, name = NameManager.new('sigmoid_grad'))]


sigmoid = Sigmoid()

In [ ]:

class Tanh(Op):
    """Tanh element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:

        args = self._ts(args)
        result = np.tanh(args[0].numpy())
        return Tensor(result, parents=args, name=NameManager.new('tanh'), operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:

        # Derivada de tanh = 1 - tanh^2(x)
        y = self(args[0].numpy())
        y_quadrado = square(y)

        result_tanh_grad = (1 - y_quadrado.numpy()) * back_grad.numpy()

        return [Tensor(result_tanh_grad,name = NameManager.new('tanh_grad'))]



tanh = Tanh()

In [ ]:

class Softmax(Op):
    """Softmax de um array de valores. Lembre-se que cada elemento do array influencia o resultado da função para todos os demais elementos."""
    def __call__(self, *args, **kwargs) -> Tensor:

        args = self._ts(args)
        exps = np.exp(args[0].numpy() - np.max(args[0].numpy(), axis = 0,keepdims=True)) # previne que e^x exploda para valores grandes

        result = exps / np.sum(exps,axis=0,keepdims=True)
        return Tensor(result, parents=args, name=NameManager.new('softmax'), operation=self)

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:

         # dL/dx = y * (dL/dy - my_sum(dL/dy * y))
        y = self(args[0].numpy())

        dot_product = np.sum(back_grad.numpy() * y.numpy())
        sub_result = back_grad.numpy() - dot_product

        result_softmax_grad = y.numpy() * sub_result

        return [Tensor(result_softmax_grad,name = NameManager.new('softmax_grad'))]


softmax = Softmax()

## Testes Básicos

Estes testes avaliam se a derivada da função está sendo calculada corretamente, mas em muitos casos **não** avaliam se os gradientes backpropagados estão sendo incorporados corretamente. Esta avaliação será feita nos problemas da próxima seção.

Operador de Soma

In [ ]:
# add

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = add(a, b)
d = add(c, 3.0)
d.backward()

# esperado: matrizes coluna contendo 1
print(a.grad)
print(b.grad)


Tensor([[1.]
 [1.]
 [1.]], name=in_grad, shape=(3, 1))
Tensor([[1.]
 [1.]
 [1.]], name=in_grad, shape=(3, 1))


Operador de Subtração

In [ ]:
# sub

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = sub(a, b)
d = sub(c, 3.0)
d.backward()

# esperado: matrizes coluna contendo 1 e -1
print(a.grad)
print(b.grad)


Tensor([[1.]
 [1.]
 [1.]], name=in_grad, shape=(3, 1))
Tensor([[-1.]
 [-1.]
 [-1.]], name=in_grad, shape=(3, 1))


Operador de Produto

In [ ]:
# prod

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = prod(a, b)
d = prod(c, 3.0)
d.backward()

# esperado: [12, 15, 18]^T
print(a.grad)
# esperado: [3, 6, 9]^T
print(b.grad)


Tensor([[12.]
 [15.]
 [18.]], name=in_grad, shape=(3, 1))
Tensor([[3.]
 [6.]
 [9.]], name=in_grad, shape=(3, 1))


Operadores trigonométricos

In [ ]:
# sin e cos

a = Tensor([np.pi, 0, np.pi/2])
b = sin(a)
c = cos(a)
d = my_sum(add(b, c))
d.backward()

# esperado: [-1, 1, -1]^T
print(a.grad)

Tensor([[-1.]
 [ 1.]
 [-1.]], name=in_grad, shape=(3, 1))


In [ ]:
# Sum

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = add(prod(a, 3.0), a)
c = my_sum(b)
c.backward()

# esperado: [4, 4, 4, 4]^T
print(a.grad)


Tensor([[4.]
 [4.]
 [4.]
 [4.]], name=in_grad, shape=(4, 1))


In [ ]:
# Mean

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = mean(a)
b.backward()

# esperado: [0.25, 0.25, 0.25, 0.25]^T
print(a.grad)


Tensor([[0.25]
 [0.25]
 [0.25]
 [0.25]], name=in_grad, shape=(4, 1))


In [ ]:
# Square

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = square(a)

# esperado: [9, 1, 0, 4]^T
print(b)

b.backward()

# esperado: [6, 2, 0, 4]
print(a.grad)

Tensor([[9.]
 [1.]
 [0.]
 [4.]], name=square:0, shape=(4, 1))
Tensor([[6.]
 [2.]
 [0.]
 [4.]], name=in_grad, shape=(4, 1))


In [ ]:
# matmul

W = Tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]
])

v = Tensor([1.0, 2.0, 3.0])

z = matmul(W, v)

# esperado: [14, 32, 50]^T
print(z)

z.backward()

# esperado:
# [1, 2, 3]
# [1, 2, 3]
# [1, 2, 3]
print(W.grad)

# esperado: [12, 15, 18]^T
print(v.grad)


Tensor([[14.]
 [32.]
 [50.]], name=matmul:0, shape=(3, 1))
Tensor([[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]], name=in_grad, shape=(3, 3))
Tensor([[12.]
 [15.]
 [18.]], name=in_grad, shape=(3, 1))


In [ ]:
# Exp

v = Tensor([1.0, 2.0, 3.0])
w = exp(v)

# esperado: [2.718..., 7.389..., 20.085...]^T
print(w)

w.backward()

# esperado: [2.718..., 7.389..., 20.085...]^T
print(v.grad)

Tensor([[ 2.71828183]
 [ 7.3890561 ]
 [20.08553692]], name=exp:0, shape=(3, 1))
Tensor([[ 2.71828183]
 [ 7.3890561 ]
 [20.08553692]], name=in_grad, shape=(3, 1))


In [ ]:
# Relu

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = relu(v)

# esperado: [0, 0, 1, 3]^T
print(w)

w.backward()

# esperado: [0, 0, 1, 1]^T
print(v.grad)

Tensor([[0.]
 [0.]
 [1.]
 [3.]], name=relu:0, shape=(4, 1))
Tensor([[0.]
 [0.]
 [1.]
 [1.]], name=in_grad, shape=(4, 1))


In [ ]:
# Sigmoid

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = sigmoid(v)

# esperado: [0.268.., 0.5, 0.731.., 0.952..]^T
print(w)

w.backward()

# esperado: [0.196..., 0.25, 0.196..., 0.045...]^T
print(v.grad)

Tensor([[0.26894142]
 [0.5       ]
 [0.73105858]
 [0.95257413]], name=sigmoid:0, shape=(4, 1))
Tensor([[0.19661193]
 [0.25      ]
 [0.19661193]
 [0.04517666]], name=in_grad, shape=(4, 1))


In [ ]:
# Tanh

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = tanh(v)

# esperado: [[-0.76159416, 0., 0.76159416, 0.99505475]^T
print(w)

w.backward()

# esperado: [0.41997434, 1., 0.41997434, 0.00986604]^T
print(v.grad)

Tensor([[-0.76159416]
 [ 0.        ]
 [ 0.76159416]
 [ 0.99505475]], name=tanh:0, shape=(4, 1))
Tensor([[0.41997434]
 [1.        ]
 [0.41997434]
 [0.00986604]], name=in_grad, shape=(4, 1))


In [ ]:
# Softmax

x = Tensor([-3.1, 0.5, 1.0, 2.0])
y = softmax(x)

# esperado: [0.00381737, 0.13970902, 0.23034123, 0.62613238]^T
print(y)

# como exemplo, calcula o MSE para um target vector
diff = sub(y, [1, 0, 0, 0])
sq = square(diff)
a = mean(sq)

# esperado: 0.36424932
print("MSE:", a)

a.backward()

# esperado: [-0.00278095, -0.02243068, -0.02654377, 0.05175539]^T
print(x.grad)



Tensor([[0.00381737]
 [0.13970902]
 [0.23034123]
 [0.62613238]], name=softmax:0, shape=(4, 1))
MSE: Tensor([[0.36424932]], name=mean:1, shape=(1, 1))
Tensor([[-0.00278095]
 [-0.02243068]
 [-0.02654377]
 [ 0.05175539]], name=in_grad, shape=(4, 1))
